<a href="https://colab.research.google.com/github/burakericok/hard_sphere_crits/blob/main/database_hard_spheres.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Critical Points of Configuration Spaces of Hard Spheres**

This is a database that contains the critical configurations of hard spheres on the rhombic dodecahedron, where the number of hard spheres is $n = 1 \ldots 12$. A critical configuration is one where spheres contacts prevent any collective motion that would allow the radius of all spheres to increase linearly with the extent of the motion. Equivalently, a critical configuration is one that has a mechanically-balanced contact graph. The torus is the rhombic dodecahedron of edge length $3/({2\sqrt{6}})$ with opposite faces identified.

## **Security Statement**
Connecting to a Jupyter notebook server running on your local machine can provide many benefits. With these benefits come serious potential risks. By connecting to a local runtime, you are allowing the Colaboratory frontend to execute code in the notebook using the local resources on your machine. This means that the notebook could:

- Invoke arbitrary commands (e.g. "rm -rf /")
- Access the local file system
- Run malicious content on your machine

Before attempting to connect to a local runtime, make sure you trust the authors of the notebook and ensure you understand what code is being executed. For more information on the Jupyter notebook server's security model, consult [Jupyter's documentation](https://jupyter-notebook.readthedocs.io/en/stable/security.html).

## **Usage**
The capabilities of this notebook are contained in two functions, `import_database` and `plot_crit`. These are defined in the `Define necessary functions` cell (double clicking expands the cell if you would like to examine the code). Run this cell first.

The next cell imports the database of critical points for the desired number of spheres, passed as the single argument to `import_database`. Critical point databases are available for $n=2 \ldots 5$. Running this cell displays the (truncated) table, which you can search for particular values of the index or radius by pressing the `Filter` button. The truncated columns contain information about the positions of the spheres and the contacts between the spheres.

The next cell displays the adjacency matrix and sphere configuration of a selected critical point, passed as the second argument to `plot_crit`. The value of this argument must be an integer that appears in the `crit_number` column of the preceeding data table.

For example, to plot the second critical point for five spheres:
- Run the `Define necessary functions` cell (only needs to be done once)
- Change the argument of `import_database` in the next cell to `5` and run the cell 
- Change the second argument of `plot_crit` in the next cell to `2` and run the cell

## **References**
Research into the topology of the configuration space of hard spheres is ongoing, but the references below provide some context for this project.

- Y. Baryshnikov, P. Bubenik, M. Kahle, [Min-type Morse theory for configuration spaces of hard spheres](https://doi.org/10.1093/imrn/rnt012), International Mathematics Research Notices 2014.9 (2014): 2577–2592.


## **Contributors**
The contributors to this project are (by alphabetical order of last name):

- Ozan Ericok (oericok@ucdavis.edu)
- Kashyap Ganesan (kganesan@ucdavis.edu)
- Jeremy Mason (jkmason@ucdavis.edu)

Please contact Jeremy Mason (jkmason@ucdavis.edu) for more information.

## **License**
The functions included in this archive are licenced under the [GNU General
Public License, Version 3](https://www.gnu.org/licenses/gpl-3.0.en.html).

## **Acknowledgements**
This material is based upon work supported by the National Science Foundation under Grant No. 1839370. Any opinions, findings, and conclusions or recommendations expressed in this material are those of the author(s) and do not necessarily reflect the views of the National Science Foundation.

In [21]:
#@title Define necessary functions
#Defines the import_database function
import pandas as pd
from google.colab import data_table

def import_database(n_spheres):
  if n_spheres < 10:
    filename = 'https://raw.githubusercontent.com/burakericok/hard_sphere_crits/main/' + str(n_spheres) + 'sphere.csv'
  else:
    filename = 'https://media.githubusercontent.com/media/burakericok/hard_sphere_crits/main/' + str(n_spheres) + 'sphere.csv'
  crit_database = pd.read_csv(filename)
  return crit_database

#Defines the plot_crit function
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
import numpy as np
import numpy.matlib
import operator

def plot_crit(database, id):
  row = database.loc[database['crit_number'] == id];
  coords = row[row.columns[row.columns.to_series().str.contains('coords')]]
  radius = row.iloc[0]['radius']
  contact_graph = row[row.columns[row.columns.to_series().str.contains('contact_graph')]]

  # constants
  n_spheres = np.int(np.size(coords)/3);

  phi = np.linspace(0, 2*np.pi)
  theta = np.linspace(-np.pi/2, np.pi/2)
  phi, theta = np.meshgrid(phi, theta)

  x_ref = np.cos(theta) * np.sin(phi) 
  y_ref = np.cos(theta) * np.cos(phi)
  z_ref = np.sin(theta)
  n0 = np.size(x_ref,0)
  n1 = np.size(x_ref,1)

  epsilon = 1e-8
  inc = 0
  cmap = plt.get_cmap("Pastel1")
  colorscale = [];
  for i in range(9):
    if n_spheres > 1:
      colorscale.append([inc/(n_spheres-1), 'rgb' + str(tuple(map(operator.sub, cmap(i)[0:3], (epsilon,epsilon,epsilon))))])
    else:
      colorscale.append([inc/1, 'rgb' + str(tuple(map(operator.sub, cmap(i)[0:3], (epsilon,epsilon,epsilon))))])
    inc = inc + 1
  cmap = plt.get_cmap("Pastel2")
  for j in range(8):
    if n_spheres > 1:
      colorscale.append([inc/(n_spheres-1), 'rgb' + str(tuple(map(operator.sub, cmap(j)[0:3], (epsilon,epsilon,epsilon))))])
    else:
      colorscale.append([inc/1, 'rgb' + str(tuple(map(operator.sub, cmap(j)[0:3], (epsilon,epsilon,epsilon))))])
    inc = inc + 1
  colorscale = colorscale[0:n_spheres]
      
  v1 = np.array([1., 0., 0.]);
  v2 = np.array([0., 1., 0.]);
  v3 = np.array([0.5, 0.5, -1. / np.sqrt(2.)]);
  v4 = np.array([-0.5, 0.5, -1. / np.sqrt(2.)]);
  v5 = np.array([0.5, -0.5, -1. / np.sqrt(2.)]);
  v6 = np.array([-0.5, -0.5, -1. / np.sqrt(2.)]);
  v7 = v3 + v6;
  v8 = v1 + v2;
  v9 = v1 - v2;
  shift = [v1, -v1, v2, -v2, v3, -v3, v4, -v4, v5, -v5, v6, -v6, v7, -v7, v8, -v8, v9, -v9];

  # Compute the adjacency matrix
  adjacency_matrix = np.zeros((n_spheres,n_spheres)) 
  adjacency_matrix[np.triu_indices(n_spheres,1)] = contact_graph
  adjacency_matrix = adjacency_matrix + adjacency_matrix.transpose()
  print('adjacency_matrix:\n',adjacency_matrix)

  # Generate the images
  ximages = np.zeros((np.size(shift,0)+1,3*n_spheres));
  ximages[0] = coords;
  for i in range(np.size(shift,0)):
      ximages[i+1] = coords + np.matlib.repmat(shift[i],1,n_spheres)
 

  # plotting - hard spheres
  if n_spheres > 1:
    bool_num = (n_spheres-1)
  else:
    bool_num = 1
  surfs = [];
  for i in range(n_spheres): 
      for j in range(np.int(np.size(ximages,0))):
          if j==0:
              alpha = 1.0
          else:
              alpha = 0.1
          sx = x_ref * radius + ximages[j,3*i]   * np.ones((n0,n1));
          sy = y_ref * radius + ximages[j,3*i+1] * np.ones((n0,n1));
          sz = z_ref * radius + ximages[j,3*i+2] * np.ones((n0,n1));
          surfs.append(
              go.Surface(
                  z = sz, 
                  x = sx, 
                  y = sy, 
                  cmin = 0, 
                  cmax = 1, 
                  colorscale = colorscale, 
                  surfacecolor = i / bool_num * np.ones((n0,n1)), 
                  showscale = False,
                  opacity = alpha,
              )
          )

  fig1 = go.Figure(data=surfs)
  fig1.update_layout(
    margin = dict(l=20, r=20, t=20, b=20),
    scene = dict(
      xaxis = dict(nticks=4, range=[-2.,2.],),
      yaxis = dict(nticks=4, range=[-2.,2.],),
      zaxis = dict(nticks=4, range=[-2.,2.],),
    ),
    scene_aspectmode='cube'
  )
  fig1.show()






  # plotting - edges
  ximages2 = np.reshape(ximages,(np.int(np.size(ximages,0)*n_spheres),3))
  d = np.zeros((np.int(np.size(ximages2,0)*(np.size(ximages2,0)-1)/2),3));
  inc = 0;
  i = 0;
  while (i<np.size(ximages2,0)):
      j=i+1;
      while(j<np.size(ximages2,0)):
          d[inc] = [i, j, np.linalg.norm(ximages2[i]-ximages2[j])];
          inc = inc+1;
          j=j+1;
      i = i+1;
          
  edge_list = d[d[:,2] < 2.001*radius,:];
  pairs = edge_list[:,[0,1]];

  #create the coordinate list for the lines
  x_lines = list()
  y_lines = list()
  z_lines = list()
  for p in pairs:
      for i in range(2):
          x_lines.append(ximages2[np.int(p[i]),0])
          y_lines.append(ximages2[np.int(p[i]),1])
          z_lines.append(ximages2[np.int(p[i]),2])
      x_lines.append(None)
      y_lines.append(None)
      z_lines.append(None)
    
  traces = []
  for i in range(n_spheres):   
      traces.append(
          go.Scatter3d(
              x = ximages[:,3*i],
              y = ximages[:,3*i+1],
              z = ximages[:,3*i+2],
              mode = 'markers',
              marker = dict(
                  size = 8,
                  color = colorscale[i][1],              
                  colorscale = colorscale,   
                  cmin = 0.,
                  cmax = 1.,
                  opacity = 1.0
              ),
              showlegend = False
          )
      )
      
  traces.append(
      go.Scatter3d(
          x = x_lines,
          y = y_lines,
          z = z_lines,
          mode = 'lines',
          line = dict(
              color = 'black',
              width = 2
          ),
          showlegend = False
      )
  )

  # Random plots for legend
  for i in range(n_spheres):
    traces.append(
      go.Scatter3d(
        x = [100], 
        y = [100],
        z = [100],
        mode = 'markers',
        marker = dict(
            size = [10],
            color = colorscale[i][1]
        ),
        name = 'Sphere ' + str(i+1)
      )
    )

  fig2 = go.Figure(data=traces)
  fig2.update_layout(
    margin = dict(l=20,r=20, t=20, b=20),
    scene = dict(
      xaxis = dict(nticks=4, range=[-2.,2.],),
      yaxis = dict(nticks=4, range=[-2.,2.],),
      zaxis = dict(nticks=4, range=[-2.,2.],),
    )
  )
  fig2.show()

In [ ]:
crit_database = import_database(9)
data_table.DataTable(crit_database, include_index=False, num_rows_per_page=10, max_columns=3, min_width='500px')

In [ ]:
plot_crit(crit_database, 1)